In [1]:
# Importing Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random

#### data set

In [2]:
df = pd.read_csv("bank.csv")

In [3]:
print("Shape : ", df.shape)
df.head()

Shape :  (5000, 14)


,ID,Age,Experience,Income,ZIP Code,Family,CCAvg,Education,Mortgage,Personal Loan,Securities Account,CD Account,Online,CreditCard
0,1,25,1,49,91107,4,1.6,1,0,0,1,0,0,0
1,2,45,19,34,90089,3,1.5,1,0,0,1,0,0,0
2,3,39,15,11,94720,1,1.0,1,0,0,0,0,0,0
3,4,35,9,100,94112,1,2.7,2,0,0,0,0,0,0
4,5,35,8,45,91330,4,1.0,2,0,0,0,0,0,1


#### Data preprocessing

In [6]:
# Deleting Columns which are not necessary
df.drop(["ID", "ZIP Code"],axis=1,inplace=True)
print(df["Experience"].unique())
df["Experience"] = abs(df["Experience"])

KeyError: "['ID', 'ZIP Code'] not found in axis"

In [7]:
df.columns

Index(['Age', 'Experience', 'Income', 'Family', 'CCAvg', 'Education',
       'Mortgage', 'Personal Loan', 'Securities Account', 'CD Account',
       'Online', 'CreditCard'],
      dtype='object')

In [8]:
df = df[['Age', 'Experience', 'Income', 'Family', 'CCAvg','Education', 'Mortgage', 'Securities Account','CD Account', 'Online', 'CreditCard', 'Personal Loan']]

In [9]:
X = df.iloc[:,:-1].values
Y = df.iloc[:,-1].values

### Spliting the data

In [10]:
x,y = X,Y

In [25]:
from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.25,random_state=0)

In [26]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

In [27]:
x_train.shape, y_train.shape

((3750, 11), (3750,))

# PyTorch 

In [28]:
import torch
from torch.utils.data import DataLoader, TensorDataset

In [29]:
BATCH_SIZE = 32
LEARNING_RATE = 0.003
EPOCH = 50

In [30]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cpu'

In [31]:
train_x = torch.from_numpy(x_train).to(torch.float32)
train_y = torch.from_numpy(y_train).to(torch.float32)

In [32]:
train_x.shape, train_y.shape

(torch.Size([3750, 11]), torch.Size([3750]))

In [33]:
data = TensorDataset(train_x,train_y)
data = DataLoader(data,batch_size=BATCH_SIZE,shuffle=True)

## model

In [34]:
class Model(torch.nn.Module):
    
    def __init__(self):
        super(Model,self).__init__()
        
        self.layer1 = torch.nn.Linear(11,16)
        self.layer2 = torch.nn.Linear(16,1)
        self.sigmoid = torch.nn.Sigmoid()
        self.relu = torch.nn.ReLU()
        
    def forward(self, x):
        x = self.layer1(x)
        x = self.relu(x)
        x = self.layer2(x)
        x = self.sigmoid(x)
        return x

In [35]:
model = Model()
# model.to(device)
print(model)

Model(
  (layer1): Linear(in_features=11, out_features=16, bias=True)
  (layer2): Linear(in_features=16, out_features=1, bias=True)
  (sigmoid): Sigmoid()
  (relu): ReLU()
)


##  Loss function and Optimizer

In [36]:
loss_function = torch.nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(),lr=LEARNING_RATE)

## Training

In [37]:
training_loss = [0]*EPOCH
training_accuracy = [0]*EPOCH

for i in range(1,EPOCH+1):
    epoch_loss = 0
    accuracy = 0
    for x_batch,y_batch in data:
        # x_batch,y_batch = x_batch.to(device),y_batch.to(device)
        optimizer.zero_grad()
        
        y_pred = model(x_batch)
        
        loss = loss_function(y_pred,y_batch.unsqueeze(1))
        loss.backward()
        
        optimizer.step()
        epoch_loss += loss.item() * len(x_batch)
        cnt = (torch.where(y_pred>=0.5, 1, 0) == y_batch.unsqueeze(1)).sum().float()
        accuracy += cnt.item()
    if(i%10 == 0):
        print(i,epoch_loss/len(data.dataset),(accuracy)/len(data.dataset))

10 0.06484903400888045 0.9781333333333333
20 0.05354246261914571 0.9810666666666666
30 0.0468826295375824 0.9837333333333333
40 0.042574787889110545 0.9861333333333333
50 0.040005037210385 0.9872


## Testing 

In [38]:
test_x = torch.from_numpy(x_test).to(torch.float32)
test_y = torch.from_numpy(y_test).to(torch.float32)

In [39]:
test = TensorDataset(test_x,test_y)
test = DataLoader(test,batch_size=1)

In [40]:
y_pred = model(test_x)
y_pred = torch.where(y_pred>=0.5, 1, 0).flatten()
accuracy = (y_pred == test_y).sum().float().item() / len(test.dataset)
print(accuracy)

0.9832


In [41]:
from sklearn.metrics import classification_report
py_report=classification_report(y_pred,test_y)
print(py_report)

              precision    recall  f1-score   support

           0       1.00      0.98      0.99      1157
           1       0.83      0.97      0.90        93

    accuracy                           0.98      1250
   macro avg       0.92      0.98      0.94      1250
weighted avg       0.99      0.98      0.98      1250



## Genetic Algorithm

In [42]:
BATCH_SIZE = 1

In [43]:
data = TensorDataset(train_x,train_y)
data = DataLoader(data,batch_size=BATCH_SIZE,shuffle=True)

In [44]:
# Accuracy 
def fitness_function(model):
    
    y_pred = model(train_x)
    y_pred = torch.where(y_pred>=0.5, 1, 0).flatten()
    accuracy = (y_pred == train_y).sum().float().item() / len(data.dataset)
    
    return accuracy

In [45]:
# Crossover and Mutation
def crossover_mutation(model1,model2):
    # Shape of the weights and biases
    shp = [i.numpy().shape for i in model1.parameters()]
    sz = [i[0]*i[1] if len(i) == 2 else i[0] for i in shp]
    
    # Flattening the parameters for cross over
    matrix_1 = np.concatenate([i.numpy().flatten() for i in model1.parameters()])
    matrix_2 = np.concatenate([i.numpy().flatten() for i in model2.parameters()])
    
    # Crossover
    start = len(matrix_1)//2 - 10
    end = len(matrix_1)//2 + 10
    mid_value = random.randrange(start,end)
    
    main_child_1 = np.concatenate([matrix_1[:mid_value], matrix_2[mid_value:]])
    main_child_2 = np.concatenate([matrix_2[:mid_value], matrix_1[mid_value:]])
    
    # Child 1 Mutation
    random_start = random.randrange(0,len(matrix_1)//2)
    random_end = random.randrange(random_start,len(matrix_1))
    
    child_1_mutate = main_child_1.copy()
    child_1_mutate[random_start:random_end] = child_1_mutate[random_start:random_end][::-1]
    
    # Child 2 Mutation
    random_start = random.randrange(0,len(matrix_1)//2)
    random_end = random.randrange(random_start,len(matrix_1))
    
    child_2_mutate = main_child_2.copy()
    child_2_mutate[random_start:random_end] = child_2_mutate[random_start:random_end][::-1]
    
    # Converting the array to parameters
    children = [main_child_1, main_child_2, child_1_mutate, child_2_mutate]
    output = list()
    
    for child in children:
        param = list()
        cum_sum = 0
        for i in range(len(sz)):
            array = child[cum_sum : cum_sum + sz[i]]
            array = array.reshape(shp[i])
            cum_sum += sz[i]
            param.append(array)
        param = np.array(param, dtype="object")
        output.append(param)
    
    output = np.array(output, dtype="object")
    return output

In [46]:
# Training 

torch.manual_seed(69)
torch.set_grad_enabled(False)
population_size = 10

def train(no):
    # Initial Population
    population = np.array([Model() for i in range(population_size)])
    
    best_model = None
    
    for loop in range(no):
        # Sort population based up on fitness function
        population = population[np.argsort([fitness_function(model) for model in population])]
        
        # Printing Max Accuracy
        best_model = population[-1]
        if(loop%10 == 0):
            print("Gen", loop, " :", fitness_function(population[-1]))
        
        # Parent Selection
        parent_max_1 = population[-1]
        parent_max_2 = population[-2]
        parent_max_3 = population[-3]
        parent_max_4 = population[-4] 
        parent_1 = population[0]
        parent_2 = population[1]
        
        # Crossover
        output_1 = crossover_mutation(parent_max_1, parent_max_3)
        output_2 = crossover_mutation(parent_max_2, parent_max_4)
        output = np.concatenate([output_1, output_2])
        
        # New population
        new_population = np.array([Model() for i in range(len(output))])
        for count, model in enumerate(new_population, 0):
            for index, param in enumerate(model.parameters(), 0):
                param.data = (torch.tensor(output[count][index]))
        
        # Add Poor performing parent to the new population
        new_population = np.concatenate([new_population, [parent_1, parent_2]])
        
        # New population becomes your next population
        population = new_population.copy()
        
    return best_model

In [47]:
best = train(100)

Gen 0  : 0.8197333333333333
Gen 10  : 0.9008
Gen 20  : 0.9053333333333333
Gen 30  : 0.9146666666666666
Gen 40  : 0.9162666666666667
Gen 50  : 0.9170666666666667
Gen 60  : 0.9205333333333333
Gen 70  : 0.9226666666666666
Gen 80  : 0.9288
Gen 90  : 0.9290666666666667


## Testing

In [48]:
y_pred = best(test_x)
y_pred = torch.where(y_pred>=0.5, 1, 0).flatten()
ga_report=classification_report(y_pred,test_y)
print(ga_report)

              precision    recall  f1-score   support

           0       0.99      0.94      0.97      1203
           1       0.38      0.87      0.53        47

    accuracy                           0.94      1250
   macro avg       0.69      0.91      0.75      1250
weighted avg       0.97      0.94      0.95      1250



## Particle Swarm optimization

In [49]:
from pyswarms.single import GlobalBestPSO
torch.manual_seed(420)
torch.set_grad_enabled(False)

model = Model()
param = np.concatenate([i.numpy().flatten() for i in model.parameters()])
shape = [i.numpy().shape for i in model.parameters()]
size = [i[0]*i[1] if len(i) == 2 else i[0] for i in shape]

print("Dim : ", len(param))
print("Layers Shape : ", shape)
print("Layers Size : ", size)

Dim :  209
Layers Shape :  [(16, 11), (16,), (1, 16), (1,)]
Layers Size :  [176, 16, 16, 1]


In [50]:
def objective_function(particle,shape=shape,size=size):
    accuracy = []
    output = []
    
    # Reshape the vector to weights and biases dimention
    for par in particle:
        param = list()
        cum_sum = 0
        for i in range(len(size)):
            array = par[cum_sum : cum_sum + size[i]]
            array = array.reshape(shape[i])
            cum_sum += size[i]
            param.append(array)
        param = np.array(param, dtype="object")
        output.append(param)
    
    for i in range(len(output)):
        # Copy Weights and Biases
        model = Model()
        for idx,wei in enumerate(model.parameters()):
            wei.data = (torch.tensor(output[i][idx])).to(torch.float32)
        
        # Calculate Accuracy
        y_pred = model(train_x)
        y_pred = torch.where(y_pred>=0.5, 1, 0).flatten()
        acc = (y_pred == train_y).sum().float().item() / len(data.dataset)
        accuracy.append(1 - acc) # Optimization function aims to reduce the cost so (1 - accuracy)
        
    return accuracy

In [51]:
# Tunable Parameters

options = {'c1': 0.6, 'c2': 0.3, 'w': 0.1}
dim = len(param)
x_max = 1.0 * np.ones(dim)
x_min = -1.0 * x_max
bounds = (x_min, x_max)

pso = GlobalBestPSO(n_particles=500, dimensions=209, options=options, bounds=bounds)

In [52]:
best_cost, best_parameters = pso.optimize(objective_function, iters=50)
print("Accuracy : ", 1 - best_cost)

2023-04-05 20:00:08,627 - pyswarms.single.global_best - INFO - Optimize for 50 iters with {'c1': 0.6, 'c2': 0.3, 'w': 0.1}
pyswarms.single.global_best: 100%|██████████████████████████████████████████████████████████████|50/50, best_cost=0.048
2023-04-05 20:00:53,839 - pyswarms.single.global_best - INFO - Optimization finished | best cost: 0.04800000000000004, best pos: [ 0.37099155  0.42134469 -0.14054767  0.05483468  0.11749422 -0.24293899
  0.37383654 -0.12252051  0.15774228  0.06722455  0.04060722  0.18180604
  0.12209702  0.07868055 -0.29191377  0.01427306  0.19699487 -0.21392934
 -0.13676235 -0.00565993  0.30456246  0.47240958  0.42963469  0.01576371
 -0.3500184  -0.42604539  0.12080956 -0.13340314  0.37319376  0.05860598
 -0.03733801  0.48075862 -0.21245809  0.01855826 -0.27609775 -0.34052139
  0.27832181 -0.46087401  0.15217016 -0.33636875  0.28740664 -0.23673198
 -0.08078382  0.07350716  0.20945096 -0.00784266  0.07482128 -0.04333414
 -0.05196728  0.23597283 -0.0412794   0.0861

Accuracy :  0.952


In [53]:
# Converting the best vector to model

result = []
for par in [best_parameters]:
    param = list()
    cum_sum = 0
    for i in range(len(size)):
        array = par[cum_sum : cum_sum + size[i]]
        array = array.reshape(shape[i])
        cum_sum += size[i]
        param.append(array)
    param = np.array(param, dtype="object")
    result.append(param)

best_model = Model()
for idx,wei in enumerate(best_model.parameters()):
    wei.data = (torch.tensor(result[0][idx])).to(torch.float32)

# Calculate Accuracy
y_pred = best_model(train_x)
y_pred = torch.where(y_pred>=0.5, 1, 0).flatten()
acc = (y_pred == train_y).sum().float().item() / len(data.dataset)
print("Accuracy : ", acc)

Accuracy :  0.952


## Testing

In [54]:
y_pred = best_model(test_x)
y_pred = torch.where(y_pred>=0.5, 1, 0).flatten()
ps_report=classification_report(y_pred,test_y)
print(ps_report)

              precision    recall  f1-score   support

           0       0.98      0.96      0.97      1168
           1       0.58      0.77      0.66        82

    accuracy                           0.95      1250
   macro avg       0.78      0.86      0.82      1250
weighted avg       0.96      0.95      0.95      1250



## Ant Colony optimization

In [55]:
torch.manual_seed(6699)
torch.set_grad_enabled(False)

In [56]:
dummy = Model()
dummy_param = np.concatenate([i.numpy().flatten() for i in dummy.parameters()])
shape = [i.numpy().shape for i in model.parameters()]
size = [i[0]*i[1] if len(i) == 2 else i[0] for i in shape]
dim = len(dummy_param)

print("Dim : ", dim)
print("Layers Shape : ", shape)
print("Layers Size : ", size)

dummy = None
dummy_param = None

Dim :  209
Layers Shape :  [(16, 11), (16,), (1, 16), (1,)]
Layers Size :  [176, 16, 16, 1]


In [57]:
# Helper Functions

def model_to_vector(model):
    vector = np.concatenate([i.numpy().flatten() for i in model.parameters()])
    return vector
    
def vector_to_model(vector):
    param = list()
    cum_sum = 0
    for i in range(len(size)):
        array = vector[cum_sum : cum_sum + size[i]]
        array = array.reshape(shape[i])
        cum_sum += size[i]
        param.append(array)
    param = np.array(param, dtype="object")
    
    dummy_model = Model()
    for idx,wei in enumerate(dummy_model.parameters()):
        wei.data = (torch.tensor(param[idx])).to(torch.float32)
        
    return dummy_model    

def calc_accuracy(model):
    y_pred = model(train_x)
    y_pred = torch.where(y_pred>=0.5, 1, 0).flatten()
    acc = (y_pred == train_y).sum().float().item() / len(data.dataset)
    
    return acc

In [58]:
ants = 10
loops = 100
evaporation_rate = 0.2
influence_factor = 0.4

In [59]:
pheromones = np.ones(dim)
max_accuracy = 0
fittest_vector = None

for loop in range(loops):
    # Generate Solution
    paths = np.array([Model() for i in range(ants)])
    accuracy = []
    
    for ant in range(ants):
        # Flatten the weights and biases
        vector = model_to_vector(paths[ant])
        
        # Multiply with pheromones 
        vector = vector * pheromones
        
        # Calculate Accuracy and Append to the list
        model = vector_to_model(vector)
        acc = calc_accuracy(model)
        accuracy.append(acc)
        
        # Update the updated path
        paths[ant] = model
        
        # Reset
        model = None
        acc = None
        
    # Select fittest path and accuracy
    paths = paths[np.argsort(accuracy)]
    
    if accuracy[np.argmax(accuracy)] > max_accuracy:
        max_accuracy = accuracy[np.argmax(accuracy)]
        fittest_vector = model_to_vector(paths[-1])
    
    # Update pheromones
    delta = 0
    for ant in range(ants):
        # Flatten the weights and biases
        vector = model_to_vector(paths[ant])
        
        # Calculate delta
        delta += (vector - fittest_vector)*influence_factor
        
    pheromones = (1-pheromones)*evaporation_rate + delta
    
    if loop%10 == 0:
        print("Iters {} :".format(loop), calc_accuracy(paths[-1]))

Iters 0 : 0.8978666666666667
Iters 10 : 0.9008
Iters 20 : 0.9008
Iters 30 : 0.9008
Iters 40 : 0.9008
Iters 50 : 0.9008
Iters 60 : 0.9008
Iters 70 : 0.9008
Iters 80 : 0.9008
Iters 90 : 0.9008


## Testing

In [60]:
y_pred = best_model(test_x)
y_pred = torch.where(y_pred>=0.5, 1, 0).flatten()
ac
print(classification_report(y_pred,test_y))

NameError: name 'ac' is not defined

In [61]:
print(ga_report)

              precision    recall  f1-score   support

           0       0.99      0.94      0.97      1203
           1       0.38      0.87      0.53        47

    accuracy                           0.94      1250
   macro avg       0.69      0.91      0.75      1250
weighted avg       0.97      0.94      0.95      1250



In [62]:
print(py_report)

              precision    recall  f1-score   support

           0       1.00      0.98      0.99      1157
           1       0.83      0.97      0.90        93

    accuracy                           0.98      1250
   macro avg       0.92      0.98      0.94      1250
weighted avg       0.99      0.98      0.98      1250

